In [ ]:
using SciMLBase
using OrdinaryDiffEq
using ForwardDiff
import ForwardDiff: jacobian

using StaticArrays
using LinearAlgebra
using Statistics

using JLD2
using Dates

using PyPlot

In [ ]:
abstract type ODEType end
abstract type AbstractTask end

---
## Загрузка библиотек и модулей

---

In [ ]:
include("Utils\\Anon_Utils.jl")
include("Utils\\CallBack_Utils.jl")
include("Utils\\Computation_Utils.jl")
include("Tasks\\ODE_Solving_Task.jl")
include("ODEs\\BS_ODE.jl")
include("Utils\\Visualization_Utils.jl")
include("Utils\\Period_Utils.jl")
include("Utils\\Session_Utils.jl")


---

## Создание сессии для сохранения результатов

---

In [ ]:
#ses = Session("Some Session", Result_Folder_Name = "Test Results")

---

## Исходная система (3 ДУ)

---

In [ ]:
ODE_Parameters = Dict("K" => 0.4,
                         "r" => 0.1,
                         "a2" => pi/2,
                         "a4" => pi)

Sngl = BS_ODE_Sngl(ODE_Parameters)

time_span = 10000.0
ic = [2.941592653589793, 0.2, 0.4]

trans_time = 0.0
alg = DP8()
callback = "x -> sin(x[1] - 0.25)"
kwargs = Dict(:dt => 1e-1, :abstol => 1e-13, :reltol => 1e-13, :maxiters => 1e10)


ODESolve = ODESolve_Task(Sngl, time_span, ic; trans_time, alg, callback, kwargs)

In [ ]:
sol = ODESolve()

### Time Series

In [ ]:
projFunc = "projCos"
varNames = ["\\phi_1", "\\phi_2", "\\phi_3"]
title = "Time Series"
plotKwargs = Dict(:linewidth => 3)

TS_properties = Dict("projFunc" => projFunc, "varNames" => varNames, "title" => title, "kwargs" => plotKwargs)
plotTimeSeries(sol; plot_properties = TS_properties);

### Poincare

In [ ]:
#savePath = "test.png"
projFunc = "projCos"
varNames = ["\\phi_1", "\\phi_2", "\\phi_3", "\\psi_1", "\\psi_2", "\\psi_3"]
title = "Poincare"
plotKwargs = Dict(:linewidth => 1)
showEvents = true
varPairs = [[[0, 1], [0, 2], [1, 2]]]

Pcr_properties = Dict("projFunc" => projFunc,
                     "varNames" => varNames,
                     "title" => title,
                     "showEvents" => showEvents,
                     "varPairs" => varPairs,
                     "kwargs" => plotKwargs)
plotPoincare(sol, plot_properties = Pcr_properties);

### Return time

In [ ]:
#savePath = "test.png"
normFunc = "normDefault"
title = "Return Time"
plotKwargs = Dict(:linewidth => 3)

RT_properties = Dict("normFunc" => normFunc, "title" => title, "kwargs" => plotKwargs)
plotReturnTime(sol, plot_properties = RT_properties);

---

## Объединенная система (6 ДУ)

---

In [ ]:
ODE_Parameters = Dict("K" => 1.5789473684210527,
                         "r" => 0.39473684210526316,
                         "a2" => pi/2,
                         "a4" => 3.24685581148453,
                         "Eps" => 1e-1)
Couple_function = "x -> 1 - cos(x)"
#Couple_function = "x -> 1/(exp(10*cos(x)))"

Duo = BS_ODE_Duo(ODE_Parameters, Couple_function)

time_span = 50.0
ic = [2.941592653589793, 0.2, 0.4, 2.941592653589793, 0.4, 0.2]
#ic = [2.941592653589793, 1.2, 1.4, 2.941592653589793, 0.4, 0.2]
#ic = [3.278216370848396e-14,
#        4.237894430800192e-8,
#        3.141592780256459,
#        3.278216370848396e-14,
#        3.141592780256459,
#        4.237894430800192e-8]

trans_time = 14.0
alg = DP8()
callback = "x -> cos(x[1])"
kwargs = Dict(:dtmax => 1e-1, :abstol => 1e-13, :reltol => 1e-13, :maxiters => 1e10)


ODESolve = ODESolve_Task(Duo, time_span, ic; trans_time, alg, callback, kwargs)

In [ ]:
sol = ODESolve()

### Time Series

In [ ]:
#savePath = "test.png"
projFunc = "projCos"
varNames = ["\\phi_1", "\\phi_2", "\\phi_3", "\\psi_1", "\\psi_2", "\\psi_3"]
title = "Time Series"
plotKwargs = Dict(:linewidth => 3)

TS_properties = Dict("projFunc" => projFunc, "varNames" => varNames, "title" => title, "kwargs" => plotKwargs)

In [ ]:
fig = plotTimeSeries(sol; plot_properties = TS_properties);

### Poincare

In [ ]:
#savePath = "test.png"
projFunc = "projCos"
varNames = ["\\phi_1", "\\phi_2", "\\phi_3", "\\psi_1", "\\psi_2", "\\psi_3"]
title = "Poincare"
plotKwargs = Dict(:linewidth => 1)
showEvents = true
varPairs = [[[0, 1], [0, 2], [1, 2]],
            [[3, 4], [3, 5], [4, 5]],
            [[0, 3], [0, 4], [0, 5]],
            [[1, 3], [1, 4], [1, 5]],
            [[2, 3], [2, 4], [2, 5]]]

Pcr_properties = Dict("projFunc" => projFunc,
                     "varNames" => varNames,
                     "title" => title,
                     "showEvents" => showEvents,
                     "varPairs" => varPairs,
                     "kwargs" => plotKwargs)

In [ ]:
fig = plotPoincare(sol, plot_properties = Pcr_properties);

### Return time

In [ ]:
#savePath = "test.png"
normFunc = "normDefault"
title = "Return Time"
plotKwargs = Dict(:linewidth => 3)

RT_properties = Dict("normFunc" => normFunc, "title" => title, "kwargs" => plotKwargs)

In [ ]:
fig = plotReturnTime(sol, plot_properties = RT_properties);

### Activation Diagram

In [ ]:
varNames = ["\\phi_1", "\\phi_2", "\\phi_3", "\\psi_1", "\\psi_2", "\\psi_3"]
title = "Activation Diagram"

AD_properties = Dict("varNames" => varNames, "title" => title)

In [ ]:
fig = plotActivationDiagram(sol, plot_properties = AD_properties);

### Period

In [ ]:
plotNorms(sol)

In [ ]:
period = getPeriodTime(sol)

### Syncs

In [ ]:
syncs = BS_Syncs(sol)

---

## Редуцированная система (3 ДУ)

---

In [ ]:
Red = BS_ODE_Red(ODE_Parameters, Couple_function, syncs)

time_span = period
ic = [y[1] for y in sol.y[1:3]]

RedODESolve = ODESolve_Task(Red, time_span, ic; alg, kwargs)

In [ ]:
red_sol = RedODESolve()

In [ ]:
fig = plotTimeSeries(red_sol; plot_properties = TS_properties);

---

## Редуцированная система + система в вариациях (3 + 6х6 ДУ)

---

In [ ]:
custom_Jacobian = z -> jacobian((y, x) -> Duo(y, x, [], 0), zeros(6), [z; Red.Syncs(z)])

In [ ]:
Red_tg = TangentODE(Red, custom_Jacobian, 3, 6)

time_span = period
u0 = [y[1] for y in sol.y[1:3]]
Q0 = diagm(ones(6))


ODESolve = TanODESolve_Task(Red_tg, time_span, u0, Q0; alg, kwargs)

In [ ]:
tansol = ODESolve()

In [ ]:
fig = plotTimeSeries(tansol);

In [ ]:
mono = reshape([t[end] for t in tansol.y[4:39]], (6, 6))

In [ ]:
eigen(mono)